In [1]:
#dumb installs
%load_ext autoreload
%autoreload 2

import os
import astrophot as ap
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from random import randint
from pandas import read_csv
from PIL import Image
%matplotlib inline

In [ ]:
img_name = os.listdir("galaxyZoo/images/")
for img in img_name:
    image = Image.open(f"galaxyZoo/images/{img}")
    image = image.convert('L')
    image = np.array(image)
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(f"galaxyZoo/fits_images/f_{img.removesuffix('.jpg')}.fits", overwrite=True)

In [5]:
final_df = read_csv(os.path.join("galaxyZoo/please.csv")) #csv file containing the classification associated with each SDSS object
spec_list = []
class_df = final_df["gz2_class"]
for n in class_df:
    spec_list.append(n)
spec_list = [s[:2] for s in spec_list] #cuts all the weird stuff from the classification
#batch because laptop doesn't have enough memory
spec_list = spec_list[:4855]

['Sb', 'Sc', 'Sc', 'SB', 'Ei', 'Er', 'SB', 'Er', 'Ei', 'SB', 'Sc', 'Er', 'Sc', 'Se', 'Er', 'Se', 'Sc', 'SB', 'Ei', 'Ei', 'Ei', 'Sb', 'Er', 'Sc', 'Er', 'SB', 'SB', 'Ei', 'SB', 'Sc', 'Ei', 'Sb', 'Sc', 'SB', 'Sb', 'Er', 'Ec', 'Se', 'Se', 'Sb', 'Se', 'Sb', 'Sc', 'SB', 'Sc', 'Ei', 'SB', 'Sb', 'Sb', 'SB', 'Sb', 'SB', 'Sc', 'Er', 'Se', 'Ei', 'Sb', 'Er', 'Er', 'Sc', 'SB', 'Ei', 'Ei', 'Sc', 'SB', 'Ec', 'Ei', 'Er', 'Ei', 'SB', 'Er', 'Ei', 'Ei', 'SB', 'Ei', 'SB', 'Sb', 'Sb', 'Sc', 'Er', 'Ei', 'Sc', 'Sb', 'SB', 'SB', 'Sc', 'Ei', 'Sb', 'Sc', 'Ei', 'Er', 'Ei', 'SB', 'Sc', 'SB', 'Sc', 'Se', 'Er', 'Ei', 'Er', 'SB', 'Er', 'SB', 'Ei', 'Er', 'Ei', 'Se', 'SB', 'Ei', 'Sb', 'Ei', 'Er', 'Sc', 'Sc', 'Se', 'Er', 'SB', 'SB', 'SB', 'Ei', 'Er', 'Sb', 'Sc', 'Se', 'Ei', 'SB', 'Er', 'Sc', 'Ec', 'Sc', 'Er', 'SB', 'Er', 'Ei', 'Sc', 'Ei', 'Sc', 'Sc', 'Sb', 'Sc', 'Sc', 'Sc', 'Er', 'Ei', 'Er', 'SB', 'Ec', 'Ei', 'Ei', 'Sb', 'Ei', 'Ei', 'SB', 'Ec', 'Se', 'Er', 'Sc', 'Ei', 'Sc', 'Ei', 'Sc', 'Sc', 'Er', 'Sb', 'Ei', 'Sc', 'Sc

In [6]:
img_name = os.listdir("galaxyZoo/fits_images/")
count = 0
params_all = []
spiral = []
elliptical = []

for img in img_name:
    #making a target (basically data for the model to fit)
    file = fits.open(f"galaxyZoo/fits_images/{img}")
    target_data = np.array(file[0].data, dtype = np.float64)

    # target_data = target_data.T

    #target object
    target = ap.image.Target_Image(
        data = target_data,
        pixelscale = 0.396, #SDSS pixel scale
        zeropoint = 22.3, #r-band in SDSS
        variance = np.ones(target_data.shape)/1e3,
    )
    model1 = ap.models.AstroPhot_Model(
    name = f"model{count}", # every model must have a unique name
    model_type = "sersic galaxy model", # this specifies the kind of model
    #we don't need to list params because we have a target !
    target = target,
    )
    model1.initialize()
    result = ap.fit.lm.LM(model1, verbose = 0).fit() #verbose is 0 by default, the higher, the more detailed output
    dict = model1.get_state()
    n = dict["parameters"]["nodes"][3]["value"]
    
    classification = spec_list[count]
    if classification[0] == "S":
        spiral.append(n)
    elif classification[0] == "E":
        elliptical.append(n)
    params_all.append(n)

    print(count)
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
Entire Isophote was Masked! R: 197.814, PA: 45.882, q: 0.900
Entire Isophote was Masked! R: 237.376, PA: 45.882, q: 0.900
Entire Isophote was Masked! R: 237.376, PA: 45.882, q: 0.600
Entire Isophote was Masked! R: 237.376, PA: 45.882, q: 0.650
Entire Isophote was Masked! R: 237.376, PA: 45.882, q: 0.700
Entire Isophote was Masked! R: 237.376, PA: 45.882, q: 0.750
Entire Isophote was Masked! R: 237.376, PA: 45.882, q: 0.850
Entire Isophote was Masked! R: 237.376, PA: 45.882, q: 0.900


/home/codespace/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Entire Isophote was Masked! R: 164.845, PA: 45.882, q: 0.550
Entire Isophote was Masked! R: 152.599, PA: 45.882, q: 0.550
Entire Isophote was Masked! R: 169.555, PA: 45.882, q: 0.550
Entire Isophote was Masked! R: 186.510, PA: 45.882, q: 0.550
Entire Isophote was Masked! R: 220.421, PA: 45.882, q: 0.550


/home/codespace/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
3

KeyboardInterrupt: 

In [7]:
print(f"The average sersic index of spiral galaxies is {sum(spiral) / len(spiral)}")
print(f"The average sersic index of elliptical galaxies is {sum(elliptical) / len(elliptical)}")
print(f"The average sersic index is {sum(params_all) / len(params_all)}")

The average sersic index of spiral galaxies is 1.5358194919149737
The average sersic index of elliptical galaxies is 1.5810735722722504
The average sersic index is 1.555014241248085


In [9]:
print(f"The range of spiral was {max(spiral) - min(spiral)}")
print(f"min: {max(spiral)}, max: {min(spiral)}")

print(f"The range of elliptical was {max(elliptical) - min(elliptical)}")
print(f"min: {max(elliptical)}, max: {min(elliptical)}")

print(f"The range of all was {max(params_all) - min(params_all)}")
print(f"min: {max(params_all)}, max: {min(params_all)}")

The range of spiral was 7.639999996958935
min: 7.999999997152622, max: 0.36000000019368594
The range of elliptical was 7.6399933244599385
min: 7.999993324643386, max: 0.36000000018344724
The range of all was 7.639999996969174
min: 7.999999997152622, max: 0.36000000018344724
